In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 12149442
paper_name = 'hanway_romesberg_2002' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
sheet_names = ['UV','MMS']

In [9]:
original_data_list = []
for s in sheet_names:
    original_data = pd.read_excel('raw_data/uv_hits.xlsx', sheet_name=s)
    print('Original data dimensions: %d x %d' % (original_data.shape))
#     print(original_data.head())
    original_data['gene'] = original_data['ORF'].astype(str)
    original_data['gene'] = clean_orf(original_data['gene'])
    
    typo_fixes = {'NTG1D':'NTG1','TOS10': 'YGR153W'}
    original_data['gene'] = original_data['gene'].apply(lambda x: typo_fixes[x] if x in typo_fixes.keys() else x)
    
    original_data['orf'] = translate_sc(original_data['gene'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data = original_data.loc[t,:]
    
    original_data.set_index('orf', inplace=True)
    
    data_cols = [c for c in original_data.columns if c.startswith('RF')]
    original_data = original_data[data_cols].apply(pd.to_numeric, axis=1, errors='coerce')
    original_data = original_data.groupby(original_data.index).mean()
    
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 167 x 9
                              ORF  RF 90%  RF 80%  RF 70%  RF 60%  RF 50%  \
index_input                                                                 
0                  High sensitivy     NaN     NaN     NaN     NaN     NaN   
37           Moderate sensitivity     NaN     NaN     NaN     NaN     NaN   
90                Low sensitivity     NaN     NaN     NaN     NaN     NaN   

             RF 40%  RF 30%  RF 50%4pts                 gene  \
index_input                                                    
0               NaN     NaN         NaN        HIGHSENSITIVY   
37              NaN     NaN         NaN  MODERATESENSITIVITY   
90              NaN     NaN         NaN       LOWSENSITIVITY   

                             orf  
index_input                       
0                  HIGHSENSITIVY  
37           MODERATESENSITIVITY  
90                LOWSENSITIVITY  
(164, 8)
Original data dimensions: 150 x 3
                              ORF  RF 0.0

In [10]:
original_data = pd.concat(original_data_list, axis=1)

In [12]:
original_data.index.name='orf'

# Converting to scale so that wt = 0 and negative values = sensitivity
original_data = original_data-1

In [13]:
original_data.shape

(282, 10)

In [24]:
# Assuming all non-hits were tested in the other screen
original_data[original_data.isnull()] = 0

# Prepare the final dataset

In [25]:
data = original_data.copy()

In [26]:
dataset_ids = [474, 11850, 11851, 11852, 11853, 11854, 11855, 11856, 4951, 4967]
datasets = datasets.reindex(index=dataset_ids)

In [27]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [28]:
data.head()

dataset_id,474,11850,11851,11852,11853,11854,11855,11856,4951,4967
data_type,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,
YAL002W,-0.03,-0.03,-0.06,-0.04,-0.07,-0.13,-0.08,-0.03,0.00,0.00
YAL005C,-0.01,0.00,0.01,0.00,0.00,-0.10,-0.06,0.06,0.00,0.00
YAL007C,-0.01,-0.01,0.01,0.00,0.00,-0.11,-0.06,0.07,0.00,0.00
YAL008W,-0.01,0.01,0.00,0.00,-0.01,-0.11,-0.04,0.06,0.00,0.00
YAL009W,0.03,0.02,0.04,0.02,0.00,-0.09,-0.06,0.14,-0.03,-0.09


## Subset to the genes currently in SGD

In [29]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 2


In [30]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,474,11850,11851,11852,11853,11854,11855,11856,4951,4967
,data_type,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,
2,YAL002W,-0.03,-0.03,-0.06,-0.04,-0.07,-0.13,-0.08,-0.03,0.00,0.00
4,YAL005C,-0.01,0.00,0.01,0.00,0.00,-0.10,-0.06,0.06,0.00,0.00
5,YAL007C,-0.01,-0.01,0.01,0.00,0.00,-0.11,-0.06,0.07,0.00,0.00
6,YAL008W,-0.01,0.01,0.00,0.00,-0.01,-0.11,-0.04,0.06,0.00,0.00
7,YAL009W,0.03,0.02,0.04,0.02,0.00,-0.09,-0.06,0.14,-0.03,-0.09


# Normalize

In [31]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [32]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [33]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,474,11850,11851,11852,11853,11854,11855,11856,4951,4967,474,11850,11851,11852,11853,11854,11855,11856,4951,4967
,data_type,value,value,value,value,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,
2,YAL002W,-0.03,-0.03,-0.06,-0.04,-0.07,-0.13,-0.08,-0.03,0.00,0.00,-2.876794,-2.246007,-4.403108,-1.933319,-3.537734,-6.427632,-2.714565,-1.095905,0.311315,-0.030537
4,YAL005C,-0.01,0.00,0.01,0.00,0.00,-0.10,-0.06,0.06,0.00,0.00,-0.958931,-0.080215,0.640452,0.004404,0.009213,-4.926841,-2.003325,1.927165,0.311315,-0.030537
5,YAL007C,-0.01,-0.01,0.01,0.00,0.00,-0.11,-0.06,0.07,0.00,0.00,-0.958931,-0.802145,0.640452,0.004404,0.009213,-5.427104,-2.003325,2.263061,0.311315,-0.030537
6,YAL008W,-0.01,0.01,0.00,0.00,-0.01,-0.11,-0.04,0.06,0.00,0.00,-0.958931,0.641716,-0.080057,0.004404,-0.497494,-5.427104,-1.292086,1.927165,0.311315,-0.030537
7,YAL009W,0.03,0.02,0.04,0.02,0.00,-0.09,-0.06,0.14,-0.03,-0.09,2.876794,1.363647,2.801978,0.973265,0.009213,-4.426577,-2.003325,4.614338,0.285372,-2.698010


# Print out

In [34]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [35]:
from IO.save_data_to_db3 import *

In [36]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/10 [00:00<?, ?it/s]

Deleting all datasets for PMID 12149442...
Inserting the new data...


100%|██████████| 10/10 [00:04<00:00,  2.11it/s]

Updating the data_modified_on field...
